In [15]:
import csv
import requests
from bs4 import BeautifulSoup
from IPython.display import HTML
import itertools
import pandas as pd

value = input('If running for the first time, enter 1, else 2: ')
# print(value)

if value == '1': 
    firsttime = True
else: 
    firsttime = False

# firsttime = True
# print(value)
# print(firsttime)

If running for the first time, enter 1, else 2:  2


Step 1: Go on the understat website (https://understat.com) 

Step 2: Within search bar, search for the plater of your liking and click enter

Step 3: For the new webpage containing all player stats, click on URL and obtain player ID from the end of the URL. 
For e.g: Player ID for the URL: https://understat.com/player/2097 is 2097

Step 4: Enter the player ID when prompted below to obtain relevant stats

In [16]:
base_url = 'https://understat.com/player'
playerID = input('Type player ID here')
url = base_url + '/' + playerID + '/'  
print(url)

Type player ID here 2097


https://understat.com/player/2097/


In [17]:
res = requests.get(url) 
soup = BeautifulSoup(res.content, "lxml")
scripts = soup.find_all('script')

player_name = soup.find('ul', attrs = {'class':'breadcrumb'}).find_all('li')[-1].text
player_name

'Lionel Messi'

In [18]:
import json 
string_with_json_obj = ''

for el in scripts: 
    if 'groupsData' in str(el): 
        string_with_json_obj = str(el).strip()
        
ind_start = string_with_json_obj.index("('")+2
ind_end = string_with_json_obj.index("')")
json_data = string_with_json_obj[ind_start:ind_end]

json_data = json_data.encode('utf8').decode('unicode_escape')

data = json.loads(json_data)


## Cropping the Data to restrict relevant entries 
data_1 = dict(itertools.islice(data.items(),1))
# data_1.update({'name':player_name})

# teams = {} 
# for id in data.keys(): 
#     teams[id] = data_1['season']['season']
# print(data_2)
data_2 = data_1['season']

columns = data_1['season'][0].keys()
values = data_1['season'][0].values()

In [19]:
# data_2
# dataframes = {}
seasons_data = []

# for x in range(0,8):
#     for row in data_2[x]:
#         seasons_data.append(data_2[x].values())
#         break

for x in range(0,len(data_2)):
    for row in data_2[x]:
        seasons_data.append(data_2[x].values())
        break

# df = pd.DataFrame(seasons_data, columns=columns) - code that was used previously 
if firsttime:
    df = pd.DataFrame(seasons_data, columns=columns)
    df['name'] = player_name
    firsttime = False
else:
    df_temp = pd.DataFrame(seasons_data, columns=columns)
    df_temp['name'] = player_name
    df = df.append(df_temp) 
    

# df_temp = pd.DataFrame(seasons_data, columns=columns)
# df_temp['name'] = player_name
# df = df.append(df_temp) 

In [20]:
df = df[['name', 'season', 'team', 'games', 'time', 'goals', 'assists', 'shots', 
         'key_passes', 'xG', 'xA', 'npg', 'npxG', 'xGChain', 'xGBuildup', 'yellow', 'red']]


In [21]:
df

,name,season,team,games,time,goals,assists,shots,key_passes,xG,xA,npg,npxG,xGChain,xGBuildup,yellow,red
0,Anthony Martial,2021,Manchester United,5,152,1,0,4,0,1.0657398700714111,0,1,1.0657398700714111,0.5925875827670097,0.17477649450302124,0,0
1,Anthony Martial,2020,Manchester United,22,1494,4,3,42,17,7.405651165172458,2.6198029601946473,4,7.405651165172458,11.814813017845154,3.3195053301751614,0,1
2,Anthony Martial,2019,Manchester United,32,2672,17,6,80,30,13.240494780242443,4.6187766548246145,17,12.47932606190443,20.409563187509775,6.357817467302084,1,0
3,Anthony Martial,2018,Manchester United,27,1632,10,2,39,36,7.954482201486826,3.2472195653244853,9,7.193313341587782,11.796286429278553,3.600729596801102,2,0
4,Anthony Martial,2017,Manchester United,30,1589,9,5,49,33,6.301933970302343,5.435461966320872,8,5.540765132755041,13.706218170002103,5.228315457701683,1,0
5,Anthony Martial,2016,Manchester United,25,1558,4,6,42,25,3.311827577650547,3.6707763336598873,4,3.311827577650547,8.690208330750465,2.4858350846916437,2,0
6,Anthony Martial,2015,Manchester United,31,2632,11,4,57,37,7.273542748764157,6.173325615003705,11,7.273542748764157,15.577428136020899,5.890168622136116,2,0
7,Anthony Martial,2015,Monaco,3,204,0,1,1,4,0.08177592605352402,0.8064165562391281,0,0.08177592605352402,0.8234055638313293,0.4310699701309204,0,0
8,Anthony Martial,2014,Monaco,35,1819,9,3,51,12,10.844686690717936,2.3954963833093643,8,9.324496451765299,12.941917221993208,2.1136934757232666,1,0
0,Lionel Messi,2021,Paris Saint Germain,3,192,0,0,9,5,0.9788687825202942,0.4944177120923996,0,0.9788687825202942,1.2385740280151367,0.21003075316548347,0,0
